# A Practical Exploration of the Wine Quality Dataset

#### By Jordan Cairns, Chris Gao, Yingzi Jin and Chun Li
#### In fulfillment of DSCI 522 Milestone 1

## Executive Summary

Our analysis aimed to develop a predictive model to distinguish between red and white wines based on various physicochemical properties. This study employed logistic regression, a model renowned for its balance between predictive power and interpretability.

The regression result suggested that residual sugar and total sulfur dioxide had high positive coefficients, indicating a strong association with white wine, whereas density showed the most substantial negative impact, followed by alcohol and volatile acidity, suggesting these are key indicators of red wine.


The logistic regression model not only achieved high accuracy but also provided valuable insights into the features most indicative of wine type. This model can assist vintners in quality control and classification tasks. Moreover, the interpretability of the model offers a foundation for further research into wine composition and its impact on sensory attributes. Future studies might explore more complex models or delve deeper into feature engineering to enhance predictive accuracy and understanding.




# Introduction

In the intricate world of oenology, the distinction between red and white wines extends beyond color, embedding itself in the nuanced spectrum of their physicochemical properties. This project embarks on a data-driven journey to unravel these complexities by leveraging statistical models to classify wines as red or white based on their inherent characteristics. Utilizing a rich dataset that encapsulates key attributes like acidity, sugar content, sulfur dioxide levels, alcohol concentration, and more, we aim to build a predictive model that not only accurately classifies the wines but also sheds light on the influential factors that underpin this classification. Through this analysis, we intend to blend the art of winemaking with the precision of data science, offering insights that could prove valuable to vintners, sommeliers, and wine enthusiasts alike in understanding the subtle distinctions between these two celebrated categories of wine.

# Data

The dataset utilized in our project is sourced from the UCI Machine Learning Repository, specifically focusing on red and white variants of Portuguese "Vinho Verde" wine​​​​. This dataset is distinguished by its emphasis on physicochemical tests to model wine quality, capturing a range of variables that reflect the sensory and chemical composition of the wine samples. Notably, it encompasses various input features like acidity, sugar content, and alcohol levels, while the output variable relates to the sensory-driven quality rating of the wines. A unique aspect of this dataset is its exclusion of data on grape types, wine brands, or prices due to privacy and logistic constraints. This attribute frames our analysis within a context of physicochemical and sensory data, offering an opportunity to delve into wine quality assessment based on measurable attributes, free from commercial biases. The dataset's structure lends itself to both classification and regression tasks, providing a fertile ground for exploring machine learning applications in the domain of wine quality evaluation.

## Data Overview:

In [1]:
import numpy as np
import pandas as pd
import requests
import zipfile
import altair as alt
import os
import sys
from sklearn import set_config
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

sys.path.insert(0, "../src")
from helper_func_wine_classification_plot import create_wine_prediction_chart 

In [2]:
# The following blocks of code was inspired by DSCI 522 Sample Milestone 1
# Download data as zip and extract
url = "https://archive.ics.uci.edu/static/public/186/wine+quality.zip"

request = requests.get(url)
with open("../data/raw/wine+quality.zip", 'wb') as f:
    f.write(request.content)

with zipfile.ZipFile("../data/raw/wine+quality.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/raw")

In [3]:
# Reading the dataframe with delimiter set to semicolon 
df_white = pd.read_csv("../data/raw/winequality-white.csv", sep = ";")
df_red = pd.read_csv("../data/raw/winequality-red.csv", sep = ";")

In [4]:
# Creating the combined df with the prediction target column added
df_white['color'] = 'white'
df_red['color'] = 'red'
df_wine = pd.concat([df_white, df_red], ignore_index=True)

In [5]:
import pandas as pd
pd.read_csv("../data/processed/scaled_wine_train.csv", index_col='Unnamed: 0')

,standardscaler__fixed acidity,standardscaler__volatile acidity,standardscaler__citric acid,standardscaler__residual sugar,standardscaler__chlorides,standardscaler__free sulfur dioxide,standardscaler__total sulfur dioxide,standardscaler__density,standardscaler__pH,standardscaler__sulphates,standardscaler__alcohol,ordinalencoder__quality,passthrough__color
1855,0.614293,-0.733172,-0.262710,1.809466,-0.087278,2.914529,1.435073,1.147636,-0.478633,-0.543148,-0.567458,3.0,white
1963,0.147699,-0.130103,1.582193,2.360508,-0.000013,1.249248,2.158809,2.363332,-1.602069,0.336276,-1.488434,3.0,white
4066,-0.085599,0.593579,-0.331039,0.622604,0.029076,1.637814,0.781946,-0.719810,-0.353807,-1.151980,1.107045,3.0,white
4414,-0.085599,-0.431638,-0.331039,1.046483,-0.436337,-0.249505,-0.030050,-0.456409,-1.102764,-0.069612,0.855870,4.0,white
1153,-1.485383,0.322198,-1.697634,-0.882166,-0.610867,-0.249505,-0.241875,-0.912295,1.768239,-1.219628,0.102343,3.0,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4040,-0.707725,-0.612558,-0.194380,-0.818584,-0.116366,0.971701,1.223248,-0.439524,-0.041741,-0.204908,-0.902358,2.0,white
5987,3.413861,0.412658,1.513863,-0.839778,1.134431,-0.471543,-1.318652,0.894365,-1.227591,1.553940,-0.483733,4.0,red
3065,-0.163365,-1.155320,0.420588,2.084987,-0.319984,1.304757,0.075863,0.945019,-0.228981,-0.746092,-0.567458,3.0,white
4970,0.380996,2.101250,-0.672688,-0.755002,0.814460,-0.693580,-0.383091,0.472249,0.582390,-0.340204,-0.818633,2.0,red


In [6]:
# Splitting the dataframe into train and test portions with a random seed set to ensure reproducibility
np.random.seed(522)
set_config(transform_output="pandas")

# Creating the split
wine_train, wine_test = train_test_split(df_wine, train_size=0.70, stratify=df_wine["color"])

In [7]:
# Saving the processed data into folder 'processed'

# Directory path
directory = "../data/processed"

# Check if the directory exists, and create it if it doesn't
if not os.path.exists(directory):
    os.makedirs(directory)

# Saving the data into a new folder
wine_train.to_csv("../data/processed/wine_train.csv")
wine_test.to_csv("../data/processed/wine_test.csv")

## Exploratory Data Analysis

The first step of EDA is to generate some histograms to visualize the effects of all numerical variables to the type of wines. By comparing these distributions side by side, we can pinpoint which features exhibit significant variations across the two categories, thereby informing feature selection for predictive modeling. Such visual tools are invaluable as they facilitate an intuitive understanding of complex data relationships, highlight potential factors that could influence the wine's classification, and guide subsequent analytical steps in the data science workflow.




In [8]:
from layered_distri_plot import layered_distri_plot

distribution_plot = layered_distri_plot(wine_train)

distribution_plot.save('../results/figures/comparative_distribution.png')
distribution_plot

alt.RepeatChart(...)

In [9]:
# Generating histograms to visualize the effects of all numerical variables to the type of wines
numeric_cols = wine_train.columns.tolist()[:-2]

from helper_func_eda_visualization import create_numeric_hist_plots
# Generating histograms to visualize the effects of all numerical variables to the type of wines

numeric_histogram_plots = create_numeric_hist_plots(wine_train, numeric_cols)
numeric_histogram_plots.display()

numeric_histogram_plots.save('../results/figures/numeric_histogram.png')

alt.RepeatChart(...)

Visually, some features do show significant differences between red and white wines and may be particularly relevant in distinguishing between the two. In particular, the following five features stand out in the histograms and could be considered significant for predicting the color of the wine.

1. Fixed & Volatile Acidity: There's a noticeable difference in the distributions, with red wines generally exhibiting higher fixed and volatile acidity.

2. Residual Sugar: White wines display a much higher residual sugar content, which could be a strong differentiator

3. Total Sulfur Dioxide: The levels are significantly higher in white wines, suggesting this feature could be key in classification.

4. Free Sulfur Dioxide: Similar to total sulfur dioxide, this feature is also markedly higher in white wines.

5. pH value: The majority of red wines seem to have a higher overall pH values.


In [10]:
corr_data = wine_train[numeric_cols].corr(
    'spearman'
    ).corr().stack().reset_index()
# # Round to 2 decimal points
corr_data['correlation_text'] = corr_data[0].map('{:.2f}'.format) 
corr_data = corr_data.rename(columns={0: 'correlation'})

In [11]:
from cor_matrix_plot import cor_matrix_plot
corr_plot = cor_matrix_plot(corr_data)
corr_plot.save('../results/figures/correlation_matrix_heatmap.png')
corr_plot

alt.LayerChart(...)

The distribution of the plot also demonstrates the majority of the explainatory variables are not strongly corrlated. However, we do observe the correlations between variable pairs `free sulfur dioxide` and `total sulfur dioxide`, as well as `density` and `alcohol` are relatively high (absolute value exceeding 0.7). This might introduce difficulties to the model to estimate the relationship between each independent variable and the dependent variable independently

## Models and Results

In [12]:
# Build a transformer to further process the data
numeric_features = wine_train.columns.tolist()[:-2]
categorical_features = wine_train.columns.tolist()[-2:-1]
columns_to_passthrough = ['color']

wine_preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OrdinalEncoder(), categorical_features),
    ('passthrough', columns_to_passthrough)
)

wine_preprocessor.fit(wine_train)
scaled_wine_train = wine_preprocessor.transform(wine_train)
scaled_wine_test = wine_preprocessor.transform(wine_test)

scaled_wine_train.to_csv("../data/processed/scaled_wine_train.csv")
scaled_wine_test.to_csv("../data/processed/scaled_wine_test.csv")

In [13]:
# Rename 'passthrough__color' back to 'color'
scaled_wine_train.rename(columns={'passthrough__color': 'color'}, inplace=True)
scaled_wine_test.rename(columns={'passthrough__color': 'color'}, inplace=True)

# Preparing data for machine learning model
X_train = scaled_wine_train.drop(columns=['color'])
y_train = scaled_wine_train['color']

X_test = scaled_wine_test.drop(columns=['color'])
y_test = scaled_wine_test['color']

In [14]:
# Creating the DummyClassifier to get the baseline score
dummy_scores = pd.DataFrame(cross_validate(
    DummyClassifier(strategy="most_frequent"),
    X_train,
    y_train,
    return_train_score=True,
    scoring=["accuracy"]
))

dummy_scores

,fit_time,score_time,test_accuracy,train_accuracy
0,0.0,0.015622,0.753846,0.753918
1,0.0,0.000000,0.753846,0.753918
2,0.0,0.000000,0.754675,0.753711
3,0.0,0.015626,0.753575,0.753986
4,0.0,0.000000,0.753575,0.753986


In [15]:
from helper_func_model_selection import model_selection
models = model_selection("dummy", "dtree", "knn", "svm", "nb", "lr")
models

{'Dummy Classifier': DummyClassifier(random_state=123),
 'Decision Tree': DecisionTreeClassifier(random_state=123),
 'KNN': KNeighborsClassifier(),
 'RBF SVM': SVC(random_state=123),
 'Naive Bayes': BernoulliNB(),
 'Logistic Regression': LogisticRegression(max_iter=1000)}

In [16]:
# The following block of code was inspired by DSCI 571 Lab 4

results_list = []

for name, model in models.items():
    
    # Create a pipeline with a CountVectorizer and the current model
    pipeline = make_pipeline(model)
    
    # Perform cross-validation
    cv_results = cross_validate(pipeline, X_train, y_train, cv=5,
    return_train_score=True,
    scoring='accuracy',
    n_jobs=-1)
    
    # Append results for the current model to the results_list
    results_list.append({
        "model": name,
        "fit_time": np.mean(cv_results['fit_time']),
        "score_time": np.mean(cv_results['score_time']),
        "test_score": np.mean(cv_results['test_score']),
        "train_score": np.mean(cv_results['train_score']),
    })

# Create a DataFrame from the results_list
results_df = pd.DataFrame(results_list)

# Set the model name as the index
results_df.set_index('model', inplace=True)

# Show the resulting DataFrame
results_df

,fit_time,score_time,test_score,train_score
model,,,,
Dummy Classifier,0.003126,0.003126,0.753904,0.753904
Decision Tree,0.021878,0.006522,0.980206,0.999835
KNN,0.015709,0.064868,0.992743,0.994392
RBF SVM,0.045812,0.024813,0.996481,0.997251
Naive Bayes,0.020046,0.005218,0.975149,0.974928
Logistic Regression,0.018763,0.000000,0.995821,0.995876


Upon concluding our exploratory data analysis and delving into model evaluation, the results delineate an intriguing landscape of model performance. Notably, while the Decision Tree, KNN, and RBF SVM models exhibit high accuracy, with the SVM model achieving the highest test scores, the choice of model cannot rest on accuracy alone. Logistic Regression, while marginally surpassed by SVM in test score metrics, stands out for its interpretability. This model provides not only a robust predictive performance but also the capacity to glean meaningful insights from the significance and impact of each feature, as reflected by its coefficients. In light of this, we opt for Logistic Regression, valuing the interpretative clarity it offers, which is instrumental for a nuanced understanding of the variables influencing wine classification. This strategic choice harmonizes predictive strength with explanatory depth, guiding us towards actionable intelligence over mere predictive prowess




In [17]:
# Fitting the Logistic Regression model and score it on the test portion
model = LogisticRegression(max_iter=1000)
model.fit(X_train,y_train)
model.score(X_test, y_test)

0.9917948717948718

In [18]:
# Producing the table to present the marginal contribution of each feature.
reg_data = {
    'Feature Name': model.feature_names_in_,
    'Coefficient': model.coef_[0]
}

result_df = pd.DataFrame(reg_data)

result_df['Feature Name'] = result_df['Feature Name'].str.replace('standardscaler__', '')
result_df['Feature Name'] = result_df['Feature Name'].str.replace('ordinalencoder__', '')

result_df

,Feature Name,Coefficient
0,fixed acidity,0.061478
1,volatile acidity,-1.325504
2,citric acid,0.245435
3,residual sugar,3.130134
4,chlorides,-0.740894
5,free sulfur dioxide,-0.316917
6,total sulfur dioxide,2.503398
7,density,-4.824456
8,pH,-0.063158
9,sulphates,-0.358068


In [19]:
# To visualize the above result

# result_df['Wine Prediction'] = result_df['Coefficient'].apply(lambda x: 'Predicting White Wine' if x > 0 else 'Predicting Red Wine')

# chart = alt.Chart(result_df).mark_bar().encode(
#     x='Coefficient',
#     y='Feature Name',
#     color=alt.Color('Wine Prediction', 
#                     legend=alt.Legend(title="Wine Prediction"), 
#                     scale=alt.Scale(domain=['Predicting Red Wine', 'Predicting White Wine'], 
#                                     range=['red', 'peachpuff'])
#                    )
# )

chart = create_wine_prediction_chart(result_df, 'red', 'peachpuff')
chart

alt.Chart(...)

In [20]:
wine_train.head(5).to_csv("../data/test/test_data_alt_distri.cvs")

The coefficients obtained from the logistic regression model provide a quantifiable measure of the impact each feature has on the likelihood of a wine being classified as red or white. Features with positive coefficients, such as residual sugar and total sulfur dioxide, increase the probability of a wine being classified as white, as indicated by the model.classes_ array. Conversely, features with negative coefficients, such as alcohol, volatile acidity, chlorides, and notably density with the largest negative coefficient, are indicative of a wine being classified as red. The magnitude of these coefficients reveals the relative importance of each feature, with density and alcohol having the most substantial influence in the negative direction and residual sugar significantly increases the odds in favor of white wine. The feature 'quality' also plays a role, albeit a smaller one, in swaying the classification towards red wine. Overall, the model's coefficients provide a nuanced understanding of how each physicochemical characteristic tilts the balance in the complex interplay of factors that determine wine color in our dataset.

## Conclusion

The logistic regression analysis reveals expected relationships between wine characteristics and their classification as red or white. Residual sugar's positive coefficient aligns with the higher levels typically found in white wines, indicating a greater likelihood of a wine being classified as white as the sugar content increases. Similarly, the positive coefficient for sulfur dioxide corresponds with the higher concentrations in white wines. The negative coefficients for alcohol and density suggest a higher probability of wine being classified as red with increasing values, which is consistent with red wines generally having higher alcohol content. These insights highlight the intricate balance of physicochemical properties influencing wine color, reaffirming the importance of considering the context and interactions of features within the dataset when interpreting model outcomes.



Nevertheless, it's important to remember that the signs and magnitudes of coefficients in logistic regression are influenced by the scale of the features and the correlations between them. These factors can affect the interpretability of the coefficients in complex ways, especially if there is multicollinearity in the data. Therefore, while the results are plausible and show some expected trends, any surprising findings would warrant a deeper investigation into the data and the model's behavi 



# Reference

Cortez, P., Cerdeira, A., Almeida, F., Matos, T., & Reis, J. (2009). Wine Quality Dataset. UCI Machine Learning Repository. Retrieved from https://archive.ics.uci.edu/dataset/186/wine+quality

Timbers, T. (2023). Breast Cancer Predictor Python Repository. GitHub. Retrieved from https://github.com/ttimbers/breast_cancer_predictor_py/tree/v0.0.2

Mor, N. S. (2022).Wine Quality and Type Prediction from Physicochemical Properties Using Neural Networks for Machine Learning: A Free Software for Winemakers and Customer. https://osf.io/ph4cu/download.

UBC Master of Data Science. (2023). DSCI 571: Supervised Learning I. UBC GitHub. Retrieved from https://github.ubc.ca/MDS-2023-24/DSCI_571_sup-learn-1_students